In [25]:
%config IPCompleter.greedy=True

# Press Shift + Tab

In [1]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

data_1 = pd.read_excel('월간_매매가격지수_아파트.xlsx',header = 10)
data_2 = pd.read_excel('월간_전세가격지수_아파트.xlsx',header = 10)
data_3 = pd.read_csv('data1.csv',usecols=range(21),lineterminator='\n',encoding = 'euc-kr',low_memory=False) #매매
data_4 = pd.read_csv('data2.csv', encoding = 'euc-kr',low_memory=False) #전세

In [2]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2677992 entries, 0 to 2677991
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   접수연도              int64  
 1   자치구코드             int64  
 2   자치구명              object 
 3   법정동코드             int64  
 4   법정동명              object 
 5   지번구분              float64
 6   지번구분명             object 
 7   본번                object 
 8   부번                float64
 9   건물명               object 
 10  계약일               int64  
 11  물건금액(만원)          int64  
 12  건물면적(㎡)           float64
 13  토지면적(㎡)           float64
 14  층                 float64
 15  권리구분              object 
 16  취소일               float64
 17  건축년도              float64
 18  건물용도              object 
 19  신고구분              object 
 20  신고한 개업공인중개사 시군구명  object 
dtypes: float64(7), int64(5), object(9)
memory usage: 429.1+ MB


In [8]:
data_3.isna().sum().sum()

12437657

In [4]:
data_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5464665 entries, 0 to 5464664
Data columns (total 23 columns):
 #   Column     Dtype  
---  ------     -----  
 0   접수연도       int64  
 1   자치구코드      int64  
 2   자치구명       object 
 3   법정동코드      float64
 4   법정동명       object 
 5   지번구분       float64
 6   지번구분명      object 
 7   본번         float64
 8   부번         float64
 9   층          float64
 10  계약일        int64  
 11  전월세 구분     object 
 12  임대면적(㎡)    float64
 13  보증금(만원)    int64  
 14  임대료(만원)    int64  
 15  건물명        object 
 16  건축년도       float64
 17  건물용도       object 
 18  계약기간       object 
 19  신규갱신여부     object 
 20  계약갱신권사용여부  object 
 21  종전 보증금     float64
 22  종전 임대료     float64
dtypes: float64(9), int64(5), object(9)
memory usage: 958.9+ MB


In [5]:
data_4.isna().sum()

접수연도               0
자치구코드              0
자치구명               0
법정동코드             33
법정동명              35
              ...   
계약기간         4507939
신규갱신여부       4558621
계약갱신권사용여부    5340249
종전 보증금       4668243
종전 임대료       5074851
Length: 23, dtype: int64

In [9]:
data_4.isna().sum().sum()

34541081

In [27]:
df3 = data_3.copy()

grouped = df3.groupby(['자치구명', '법정동명', '본번', '부번'])

# 각 그룹의 첫 번째 '건물명'을 가져옵니다.
first_building_names = grouped['건물명'].transform('first')

# '건물명' 컬럼을 첫 번째 '건물명'으로 업데이트합니다.
df3['건물명'] = first_building_names
df3['계약월'] = (df3['계약일']%10000//100).astype('float64')
df3['건물명'] = df3['건물명'].str.strip('\t')
df3['건물면적(㎡)'] = df3['건물면적(㎡)'].map(lambda x: float(round(x)) if ~np.isnan(x)  else x)
df3['본번'] = df3['본번'].str.lstrip('0')
df3['본번'] = pd.to_numeric(df3['본번'],errors='coerce')
df3['토지면적(㎡)'] = df3['토지면적(㎡)'].map(lambda x: round(x) if ~np.isnan(x)  else x)
df3 = df3[df3['접수연도']>2010] 
df3 = df3.drop(df3[(df3['건물용도']=='연립다세대') | (df3['건물용도']=='단독다가구')].index)
df3.drop(columns=['신고구분', '신고한 개업공인중개사 시군구명', '토지면적(㎡)', '취소일','권리구분','지번구분','계약일'],inplace = True)
df3.rename(columns={"건물면적(㎡)": "면적"},inplace = True)
df3 = df3.sort_values(by=['접수연도'], ascending=True)
df3 = df3.sort_values(by=['자치구명', '법정동명', '건물명', '면적']).reset_index(drop=True)
df3['층'] = df3['층'].map(lambda x: '2층 초과' if x >2 else '2층 이하') 

df3.head()

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분명,본번,부번,건물명,물건금액(만원),면적,층,건축년도,건물용도,계약월
0,2023,11680,강남구,10300,개포동,대지,1237.0,8.0,YH빌리지,37000,28.0,2층 이하,2017.0,오피스텔,2.0
1,2022,11680,강남구,10300,개포동,대지,1237.0,8.0,YH빌리지,40000,31.0,2층 이하,2017.0,오피스텔,1.0
2,2023,11680,강남구,10300,개포동,대지,1237.0,8.0,YH빌리지,40000,37.0,2층 이하,2017.0,오피스텔,7.0
3,2011,11680,강남구,10300,개포동,대지,655.0,2.0,개포2차현대아파트(220),65500,78.0,2층 초과,1988.0,아파트,7.0
4,2011,11680,강남구,10300,개포동,대지,655.0,2.0,개포2차현대아파트(220),64000,78.0,2층 초과,1988.0,아파트,7.0


In [28]:
df3.loc[(df3['건물명']=='파크하비오(204동)'), ['건축년도']] = 2016

df3.loc[(df3['건물명']=='대명벨리온')&(df3['건물용도']=='오피스텔'), ['건축년도']] = 2017

df3.loc[(df3['건물명']=='래미안포레'), ['본번']] = 687

df3.loc[(df3['건물명']=='랑데르'), ['본번']] = 282

df3.loc[(df3['건물명']=='랑데르'), ['부번']] = 2

df3.loc[(df3['건물명']=='동대문 푸르지오시티'), ['건축년도']] = 2016

df3.loc[(df3['건물명']=='도봉 투웨니퍼스트 2단지'), ['건축년도']] = 2017

df3.loc[(df3['건물명']=='엠스테이트'), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='광진구')&(df3['법정동명']=='화양동')&(df3['본번']==111.0)&(df3['부번']==12.0), ['건물명']] = '빌리브인테라스'

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='석촌동')&(df3['본번']==24.0)&(df3['부번']==11.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='종로구')&(df3['법정동명']=='숭인동')&(df3['본번']==298.0)&(df3['부번']==9.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='가산동')&(df3['본번']==142.0)&(df3['부번']==17.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='문래동3가')&(df3['본번']==81.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='마포구')&(df3['법정동명']=='신공덕동')&(df3['본번']==1.0)&(df3['부번']==48), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='구로구')&(df3['법정동명']=='구로동')&(df3['본번']==830.0)&(df3['부번']==0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='중구')&(df3['법정동명']=='황학동')&(df3['본번']==19.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강동구')&(df3['법정동명']=='길동')&(df3['본번']==414.0)&(df3['부번']==9.0), ['건축년도']] = 2022

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='마곡동')&(df3['본번']==784.0)&(df3['부번']==9.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='서초구')&(df3['법정동명']=='내곡동')&(df3['건물명']=='서초포레스타2단지'), ['본번']] = 384

df3.loc[(df3['자치구명']=='광진구')&(df3['법정동명']=='구의동')&(df3['본번']==667.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='성동구')&(df3['법정동명']=='성수동1가')&(df3['본번']==656.0)&(df3['부번']==449.0), ['건축년도']] = 2022

df3.loc[(df3['자치구명']=='중랑구')&(df3['법정동명']=='상봉동')&(df3['본번']==129.0)&(df3['부번']==19.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='마포구')&(df3['법정동명']=='신공덕동')&(df3['본번']==5.0)&(df3['부번']==75.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='신천동')&(df3['본번']==29.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강남구')&(df3['법정동명']=='대치동')&(df3['본번']==944.0)&(df3['부번']==21.0), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='석촌동')&(df3['본번']==24.0)&(df3['부번']==14.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='관악구')&(df3['법정동명']=='봉천동')&(df3['본번']==1568.0)&(df3['부번']==1.0), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='서초구')&(df3['법정동명']=='서초동')&(df3['본번']==1487.0)&(df3['부번']==116.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강남구')&(df3['법정동명']=='삼성동')&(df3['본번']==45.0)&(df3['부번']==9.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강남구')&(df3['법정동명']=='율현동')&(df3['건물명']=='훼미리타운'), ['본번']] = 672

df3.loc[(df3['자치구명']=='강남구')&(df3['법정동명']=='율현동')&(df3['건물명']=='훼미리타운'), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강동구')&(df3['법정동명']=='강일동')&(df3['본번']==677.0)&(df3['부번']==3.0), ['건축년도']] = 2015

df3.loc[(df3['자치구명']=='강동구')&(df3['법정동명']=='천호동')&(df3['본번']==167.0)&(df3['부번']==120.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='강북구')&(df3['법정동명']=='수유동')&(df3['본번']==277.0)&(df3['부번']==33.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강북구')&(df3['법정동명']=='수유동')&(df3['본번']==279.0)&(df3['부번']==54.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='가양동')&(df3['본번']==274.0)&(df3['부번']==4.0), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='등촌동')&(df3['본번']==634.0)&(df3['부번']==10.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='마곡동')&(df3['본번']==5.0)&(df3['부번']==1.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='마곡동')&(df3['본번']==747.0)&(df3['부번']==0.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='염창동')&(df3['본번']==253.0)&(df3['부번']==6.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='염창동')&(df3['본번']==253.0)&(df3['부번']==7.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='화곡동')&(df3['본번']==920.0)&(df3['부번']==15.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='강서구')&(df3['법정동명']=='화곡동')&(df3['본번']==899.0)&(df3['부번']==8.0), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='관악구')&(df3['법정동명']=='봉천동')&(df3['본번']==919.0)&(df3['부번']==38.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='관악구')&(df3['법정동명']=='신림동')&(df3['본번']==1474.0)&(df3['부번']==4.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='광진구')&(df3['법정동명']=='화양동')&(df3['본번']==42.0)&(df3['부번']==7.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='광진구')&(df3['법정동명']=='화양동')&(df3['본번']==111.0)&(df3['부번']==22.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='구로구')&(df3['법정동명']=='개봉동')&(df3['본번']==198.0)&(df3['부번']==4.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='구로구')&(df3['법정동명']=='개봉동')&(df3['본번']==66.0)&(df3['부번']==27.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='구로구')&(df3['법정동명']=='오류동')&(df3['건물명']=='새롬스타빌'), ['본번']] = 280

df3.loc[(df3['자치구명']=='구로구')&(df3['법정동명']=='오류동')&(df3['건물명']=='새롬스타빌'), ['부번']] = 12

df3.loc[(df3['자치구명']=='구로구')&(df3['법정동명']=='항동')&(df3['건물명']=='우남퍼스트빌더센트럴'), ['본번']] = 216

df3.loc[(df3['자치구명']=='구로구')&(df3['법정동명']=='항동')&(df3['건물명']=='우남퍼스트빌더센트럴'), ['부번']] = 1

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='가산동')&(df3['본번']==143.0)&(df3['부번']==20.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='가산동')&(df3['본번']==238.0)&(df3['부번']==97.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='가산동')&(df3['본번']==238.0)&(df3['부번']==97.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='가산동')&(df3['본번']==238.0)&(df3['부번']==24.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==378.0)&(df3['부번']==464.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==1023.0)&(df3['부번']==4.0), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==378.0)&(df3['부번']==461.0), ['건축년도']] = 2015

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==1006.0)&(df3['부번']==204.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==973.0)&(df3['부번']==47.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==1006.0)&(df3['부번']==110.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==1009.0)&(df3['부번']==75.0), ['건축년도']] = 2015

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==297.0)&(df3['부번']==19.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==295.0)&(df3['부번']==9.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='독산동')&(df3['본번']==330.0)&(df3['부번']==13.0), ['건축년도']] = 2022

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='시흥동')&(df3['본번']==831.0)&(df3['부번']==5.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='시흥동')&(df3['본번']==831.0)&(df3['부번']==5.0), ['건물명']] = '은연빌라트'

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='시흥동')&(df3['본번']==995.0)&(df3['부번']==24.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='시흥동')&(df3['본번']==789.0)&(df3['부번']==15.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='시흥동')&(df3['본번']==789.0)&(df3['부번']==49.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='금천구')&(df3['법정동명']=='시흥동')&(df3['본번']==888.0)&(df3['부번']==14.0), ['건물명']] = '신영메트로시티'

df3.loc[(df3['자치구명']=='도봉구')&(df3['법정동명']=='쌍문동')&(df3['본번']==103.0)&(df3['부번']==79.0), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='도봉구')&(df3['법정동명']=='창동')&(df3['본번']==635.0)&(df3['부번']==43.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='도봉구')&(df3['법정동명']=='창동')&(df3['본번']==607.0)&(df3['부번']==25.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='도봉구')&(df3['법정동명']=='창동')&(df3['본번']==424.0)&(df3['부번']==3.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='도봉구')&(df3['법정동명']=='창동')&(df3['본번']==581.0)&(df3['부번']==19.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='도봉구')&(df3['법정동명']=='창동')&(df3['본번']==578.0)&(df3['부번']==59.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='동대문구')&(df3['법정동명']=='답십리동')&(df3['본번']==530.0)&(df3['부번']==15.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='동대문구')&(df3['법정동명']=='장안동')&(df3['본번']==106.0)&(df3['부번']==6.0), ['건축년도']] = 2022

df3.loc[(df3['자치구명']=='동대문구')&(df3['법정동명']=='장안동')&(df3['본번']==106.0)&(df3['부번']==26.0), ['건축년도']] = 2022

df3.loc[(df3['자치구명']=='동대문구')&(df3['법정동명']=='장안동')&(df3['본번']==417.0)&(df3['부번']==2.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='동대문구')&(df3['법정동명']=='장안동')&(df3['본번']==382.0)&(df3['부번']==3.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='동대문구')&(df3['법정동명']=='장안동')&(df3['본번']==382.0)&(df3['부번']==4.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='동대문구')&(df3['법정동명']=='장안동')&(df3['본번']==372.0)&(df3['부번']==8.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='동대문구')&(df3['법정동명']=='장안동')&(df3['본번']==346.0)&(df3['부번']==2.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='동작구')&(df3['법정동명']=='대방동')&(df3['본번']==20.0)&(df3['부번']==7.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='동작구')&(df3['법정동명']=='대방동')&(df3['본번']==394.0)&(df3['부번']==26.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='마포구')&(df3['법정동명']=='공덕동')&(df3['본번']==16.0)&(df3['부번']==10.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='서대문구')&(df3['법정동명']=='북가좌동')&(df3['본번']==351.0)&(df3['부번']==16.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='서초구')&(df3['법정동명']=='신원동')&(df3['건물명']=='해피트리앤'), ['본번']] = 659

df3.loc[(df3['자치구명']=='중랑구')&(df3['법정동명']=='상봉동')&(df3['본번']==127.0)&(df3['부번']==18.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='중랑구')&(df3['법정동명']=='면목동')&(df3['본번']==90.0)&(df3['부번']==32.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='중랑구')&(df3['법정동명']=='면목동')&(df3['본번']==177.0)&(df3['부번']==45.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='중랑구')&(df3['법정동명']=='면목동')&(df3['본번']==90.0)&(df3['부번']==25.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='중랑구')&(df3['법정동명']=='면목동')&(df3['본번']==117.0)&(df3['부번']==4.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='중랑구')&(df3['법정동명']=='망우동')&(df3['건물명']=='신내역 힐데스하임참좋은'), ['본번']] = 598

df3.loc[(df3['자치구명']=='중랑구')&(df3['법정동명']=='망우동')&(df3['건물명']=='신내역 금강펜테리움 센트럴파크'), ['본번']] = 610

df3.loc[(df3['자치구명']=='중구')&(df3['법정동명']=='황학동')&(df3['본번']==1153.0)&(df3['부번']==0), ['건축년도']] = 2022

df3.loc[(df3['자치구명']=='중구')&(df3['법정동명']=='황학동')&(df3['본번']==1496.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='진관동')&(df3['본번']==100.0)&(df3['부번']==1), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='응암동')&(df3['본번']==117.0)&(df3['부번']==30), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='응암동')&(df3['본번']==376.0)&(df3['부번']==42.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='응암동')&(df3['본번']==112.0)&(df3['부번']==2), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='응암동')&(df3['본번']==323.0)&(df3['부번']==8), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='응암동')&(df3['본번']==81.0)&(df3['부번']==173), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='응암동')&(df3['본번']==599.0)&(df3['부번']==15.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='응암동')&(df3['본번']==126.0)&(df3['부번']==12.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='역촌동')&(df3['본번']==39.0)&(df3['부번']==3.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='역촌동')&(df3['본번']==16.0)&(df3['부번']==18.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='역촌동')&(df3['본번']==1.0)&(df3['부번']==47.0), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='신사동')&(df3['본번']==230.0)&(df3['부번']==10.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='신사동')&(df3['본번']==230.0)&(df3['부번']==4.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='수색동')&(df3['본번']==30.0)&(df3['부번']==0), ['건축년도']] = 2021

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='불광동')&(df3['본번']==13.0)&(df3['부번']==23.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='불광동')&(df3['본번']==281.0)&(df3['부번']==50.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='불광동')&(df3['본번']==281.0)&(df3['부번']==68.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='대조동')&(df3['본번']==219.0)&(df3['부번']==3.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='녹번동')&(df3['본번']==81.0)&(df3['부번']==17.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='구산동')&(df3['본번']==304.0)&(df3['부번']==1.0), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='은평구')&(df3['법정동명']=='갈현동')&(df3['본번']==468.0)&(df3['부번']==5.0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='용산구')&(df3['법정동명']=='한강로2가')&(df3['본번']==422.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='영등포동7가')&(df3['본번']==94.0)&(df3['부번']==298), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='영등포동7가')&(df3['본번']==94.0)&(df3['부번']==355), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='영등포동6가')&(df3['본번']==42.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='영등포동1가')&(df3['본번']==57.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='양평동5가')&(df3['본번']==94.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='양평동3가')&(df3['본번']==49.0)&(df3['부번']==4), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='신길동')&(df3['본번']==449.0)&(df3['부번']==20), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='문래동3가')&(df3['본번']==55.0)&(df3['부번']==4), ['건축년도']] = 2004

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='당산동4가')&(df3['본번']==27.0)&(df3['부번']==5), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='당산동3가')&(df3['본번']==395.0)&(df3['부번']==46), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='당산동2가')&(df3['본번']==159.0)&(df3['부번']==3), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='당산동1가')&(df3['본번']==236.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='당산동')&(df3['본번']==121.0)&(df3['부번']==228), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='영등포구')&(df3['법정동명']=='당산동')&(df3['본번']==121.0)&(df3['부번']==137), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='양천구')&(df3['법정동명']=='신정동')&(df3['본번']==733.0)&(df3['부번']==53), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='양천구')&(df3['법정동명']=='신정동')&(df3['본번']==891.0)&(df3['부번']==25), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='양천구')&(df3['법정동명']=='목동')&(df3['본번']==607.0)&(df3['부번']==13), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='양천구')&(df3['법정동명']=='목동')&(df3['본번']==713.0)&(df3['부번']==14), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='장지동')&(df3['건물명']=='위례중앙푸르지오2단지(C1-6블럭)'), ['본번']] = 896

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='오금동')&(df3['본번']==80.0)&(df3['부번']==13), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='오금동')&(df3['본번']==134.0)&(df3['부번']==9), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='마천동')&(df3['본번']==42.0)&(df3['부번']==8), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='거여동')&(df3['건물명']=='휴먼빌 시티'), ['본번']] = 605

df3.loc[(df3['자치구명']=='송파구')&(df3['법정동명']=='거여동')&(df3['건물명']=='위례신도시A1-4BL 호반써밋송파2'), ['본번']] = 607

df3.loc[(df3['자치구명']=='성북구')&(df3['법정동명']=='하월곡동')&(df3['본번']==90.0)&(df3['부번']==1995), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='성북구')&(df3['법정동명']=='장위동')&(df3['본번']==218.0)&(df3['부번']==12), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='성북구')&(df3['법정동명']=='석관동')&(df3['본번']==175.0)&(df3['부번']==8), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='성북구')&(df3['법정동명']=='석관동')&(df3['본번']==175.0)&(df3['부번']==18), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='성북구')&(df3['법정동명']=='동선동4가')&(df3['본번']==169.0)&(df3['부번']==0), ['건축년도']] = 2017

df3.loc[(df3['자치구명']=='성동구')&(df3['법정동명']=='행당동')&(df3['본번']==293.0)&(df3['부번']==8), ['건축년도']] = 2018

df3.loc[(df3['자치구명']=='성동구')&(df3['법정동명']=='성수동2가')&(df3['본번']==299.0)&(df3['부번']==175), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='성동구')&(df3['법정동명']=='성수동2가')&(df3['본번']==289.0)&(df3['부번']==167), ['건축년도']] = 2016

df3.loc[(df3['자치구명']=='성동구')&(df3['법정동명']=='성수동2가')&(df3['본번']==299.0)&(df3['부번']==29), ['건축년도']] = 2016

In [29]:
df3.shape

(1168900, 15)

In [30]:
df3[df3.isna().any(axis=1)].head()

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분명,본번,부번,건물명,물건금액(만원),면적,층,건축년도,건물용도,계약월
736677,2022,11650,서초구,11100,신원동,NaN,NaN,NaN,NaN,18000,21.0,2층 초과,2015.0,오피스텔,5.0
736678,2022,11650,서초구,11100,신원동,NaN,NaN,NaN,NaN,18000,21.0,2층 초과,2015.0,오피스텔,5.0
736679,2022,11650,서초구,11100,신원동,NaN,NaN,NaN,NaN,18000,21.0,2층 초과,2015.0,오피스텔,5.0
736680,2022,11650,서초구,11100,신원동,NaN,NaN,NaN,NaN,17500,21.0,2층 초과,2015.0,오피스텔,10.0
736681,2022,11650,서초구,11100,신원동,NaN,NaN,NaN,NaN,17500,21.0,2층 초과,2015.0,오피스텔,10.0


In [31]:
df3 = df3.dropna()

df3.shape

(1168744, 15)

In [33]:
df3['평균물건금액(만원)'] = df3.groupby(['자치구명', '법정동명', '본번', '부번', '건축년도','층','면적','접수연도','계약월'])['물건금액(만원)'].transform('mean')
df3['본번'] = df3['본번'].astype(int)
df3['부번'] = df3['부번'].astype(int)
df3['면적'] = df3['면적'].astype(int)
df3['건축년도'] = df3['건축년도'].astype(int)
df3['계약월'] = df3['계약월'].astype(int)
df3['평균물건금액(만원)'] = df3['평균물건금액(만원)'].astype(int)
print(df3.shape)
df3.head()

(1168744, 16)


,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분명,본번,부번,건물명,물건금액(만원),면적,층,건축년도,건물용도,계약월,평균물건금액(만원)
0,2023,11680,강남구,10300,개포동,대지,1237,8,YH빌리지,37000,28,2층 이하,2017,오피스텔,2,37000
1,2022,11680,강남구,10300,개포동,대지,1237,8,YH빌리지,40000,31,2층 이하,2017,오피스텔,1,40000
2,2023,11680,강남구,10300,개포동,대지,1237,8,YH빌리지,40000,37,2층 이하,2017,오피스텔,7,40000
3,2011,11680,강남구,10300,개포동,대지,655,2,개포2차현대아파트(220),65500,78,2층 초과,1988,아파트,7,64750
4,2011,11680,강남구,10300,개포동,대지,655,2,개포2차현대아파트(220),64000,78,2층 초과,1988,아파트,7,64750


In [36]:
# 전세
df4 = data_4.copy()

grouped = df4.groupby(['자치구명', '법정동명', '본번', '부번'])

# 각 그룹의 첫 번째 '건물명'을 가져옵니다.
first_building_names = grouped['건물명'].transform('first')

# '건물명' 컬럼을 첫 번째 '건물명'으로 업데이트합니다.
df4['건물명'] = first_building_names
df4['건물명'] = df4['건물명'].str.strip('\t')

def months(x):
    a = x.split('~')
    if len(a[0]) == 7:
        p = dt.strptime(a[0], '%Y.%m')
    else:
        p = dt.strptime('20'+a[0], '%Y.%m')
    if len(a[1]) == 7:
        f = dt.strptime(a[1], '%Y.%m')
    else:
        f = dt.strptime('20'+a[1], '%Y.%m')
    return (f.year - p.year)*12 + f.month - p.month

df4['계약기간'] = df4['계약기간'].fillna(value = 0)
df4['계약개월'] = df4['계약기간'].map(lambda x: float(months(x)) if x != 0 else x)
df4['계약기간'] = df4['계약기간'].map(lambda x: np.nan if x == 0 else x)
df4['계약개월'] = df4['계약개월'].map(lambda x: np.nan if x == 0 else x)
df4['계약월'] = (df4['계약일']%10000//100).astype('float64')
df4['임대면적(㎡)'] = df4['임대면적(㎡)'].map(lambda x: round(x) if ~np.isnan(x)  else x)
df4.loc[(df4['전월세 구분']=='월세') & (df4['계약개월'] >= 36.0) & (df4['계약개월']*df4['임대료(만원)'] > df4['보증금(만원)']),'전월세 구분'] = '전세'
df4 = df4.loc[~((df4['전월세 구분']=='월세') & (df4['계약개월'] >= 36.0) & ((df4['계약개월']*df4['임대료(만원)'] <= df4['보증금(만원)'])))]
df4 = df4.loc[~(df4['보증금(만원)']<=3400)]
df4.drop(columns=['계약갱신권사용여부', '종전 보증금', '종전 임대료', '계약개월', '계약기간', '지번구분', '신규갱신여부','계약일'],inplace = True)
df4 = df4.loc[~(df4['전월세 구분']=='월세')]
df4 = df4.drop(df4[(df4['건물용도']=='연립다세대') | (df4['건물용도']=='단독다가구')].index)
df4.rename(columns={"임대면적(㎡)": "면적"},inplace = True)
df4 = df4.sort_values(by=['접수연도'], ascending=True)
df4 = df4.sort_values(by=['자치구명', '법정동명', '건물명', '면적']).reset_index(drop=True)
df4['층'] = df4['층'].map(lambda x: '2층 초과' if x >2 else ('2층 이하' if x<=2 else x)) 
print(df4.isna().sum())

접수연도          0
자치구코드         0
자치구명          0
법정동코드         3
법정동명          4
지번구분명       209
본번          209
부번          209
층            24
전월세 구분        0
면적           24
보증금(만원)       0
임대료(만원)       0
건물명         210
건축년도       6341
건물용도          0
계약월           0
dtype: int64


In [38]:
sett = set(df4['건물명']) & set(df3['건물명'])
df_m = df3.loc[df3['건물명'].isin(sett)]

def tc(x):
    if len(x) == 6:
        a = x.split('-')
        return '-0'.join(a)
    else:
        return x

df_m = df_m.groupby(['자치구명', '법정동명', '본번', '부번', '건물명', '건축년도', '면적', '층', '접수연도', '계약월', '건물용도'])['물건금액(만원)'].agg(['mean']).reset_index()
df_m = df_m.sort_values(['자치구명', '법정동명', '본번', '부번', '건물명', '건축년도', '면적', '층']).reset_index(drop=True)
df_m = df_m.rename(columns={'mean': '월별평균 매매가'})
df_m = df_m.loc[~((df_m['접수연도']==2023) & (df_m['계약월']>7))]
df_m['계약월'] = df_m['접수연도'].astype(str) + '-' + df_m['계약월'].astype(str)
df_m['계약월'] = df_m['계약월'].apply(lambda x: tc(x))
df_m.drop(columns=['접수연도'],inplace = True)
df_m['주소'] = df_m['자치구명'] + ' ' + df_m['법정동명'] + ' ' + df_m['본번'].astype(str) + '-' + df_m['부번'].astype(str)
cols_m = df_m.columns.tolist()
cols_m = cols_m[-1:] + cols_m[:-1]
df_m = df_m[cols_m]
df_m.drop(columns=['자치구명', '법정동명', '본번', '부번'],inplace = True)
print(df_m.shape)
display(df_m.head())

(620060, 8)


,주소,건물명,건축년도,면적,층,계약월,건물용도,월별평균 매매가
0,강남구 개포동 12-0,성원대치2단지아파트,1992,33,2층 이하,2011-02,아파트,30650.0
1,강남구 개포동 12-0,성원대치2단지아파트,1992,33,2층 이하,2011-03,아파트,30000.0
2,강남구 개포동 12-0,성원대치2단지아파트,1992,33,2층 이하,2011-06,아파트,32000.0
3,강남구 개포동 12-0,성원대치2단지아파트,1992,33,2층 이하,2011-12,아파트,28250.0
4,강남구 개포동 12-0,성원대치2단지아파트,1992,33,2층 이하,2012-01,아파트,27100.0


In [42]:
df_m['계약월'].unique()

array(['2011-02', '2011-03', '2011-06', '2011-12', '2012-01', '2012-09',
       '2013-02', '2013-09', '2013-11', '2014-01', '2014-02', '2014-03',
       '2014-07', '2014-09', '2015-02', '2015-03', '2015-04', '2015-05',
       '2015-06', '2015-07', '2015-11', '2016-03', '2016-04', '2016-05',
       '2016-09', '2016-10', '2017-04', '2017-05', '2017-07', '2017-10',
       '2018-02', '2018-08', '2020-08', '2020-09', '2020-10', '2020-12',
       '2023-01', '2023-02', '2011-01', '2011-04', '2011-05', '2011-09',
       '2011-10', '2012-02', '2012-03', '2012-05', '2012-06', '2012-07',
       '2012-08', '2012-10', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-08', '2013-10', '2013-12', '2014-04', '2014-05', '2014-06',
       '2014-08', '2014-10', '2014-11', '2014-12', '2015-01', '2015-08',
       '2015-09', '2015-10', '2015-12', '2016-01', '2016-02', '2016-06',
       '2016-07', '2016-08', '2016-11', '2016-12', '2017-02', '2017-03',
       '2017-06', '2017-08', '2017-09', '2017-11', 

In [39]:
df1 = data_1.copy()

def tc2(x):
    a = x.strip('월').split('년 ')
    return '-'.join(a)

df1.rename(columns={"Unnamed: 1": "자치구명"}, inplace=True)
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df1.drop(['지 역'], axis = 1, inplace = True)
df1.drop([0,1],inplace = True)
df1 = df1.reset_index(drop=True)
df1 = df1.rename(index = df1['자치구명'])
df1.drop(['자치구명'], axis = 1, inplace = True)
df1.columns = df1.columns.map(tc2)
df1.head()

,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07
종로구,57.297,57.709,58.077,58.108,58.225,58.454,58.439,58.788,59.637,59.966,59.96,60.231,60.231,60.644,61.451,61.502,61.507,61.709,61.513,61.666,62.257,63.128,65.881,67.128,68.05,68.552,68.791,68.947,69.051,69.485,69.728,69.794,70.022,70.172,70.275,70.735,71.267,71.612,72.25,73.71,73.848,74.24,75.244,75.776,76.748,77.191,78.351,78.076,78.01,77.832,78.235,79.055,79.137,79.353,79.281,79.56,80.613,81.242,81.143,81.22,81.358,81.498,81.56,81.495,81.438,81.46,81.268,80.973,80.94,80.903,80.934,80.879,80.879,80.96,81.446,81.45,81.341,81.428,81.328,81.787,81.786,81.794,81.713,81.685,81.685,81.685,81.685,81.685,81.685,81.642,81.642,81.759,81.695,81.46,81.411,81.408,81.408,81.408,81.337,81.314,81.242,80.971,80.785,80.627,80.596,80.759,80.819,80.858,80.873,80.97,80.979,80.927,80.887,80.826,80.808,80.825,81.042,81.077,81.076,81.046,81.064,81.09,81.284,81.664,81.895,82.099,82.25,82.409,82.554,82.684,82.923,83.014,83.055,83.068,83.094,83.138,83.262,83.418,83.649,83.933,84.099,84.386,84.704,84.867,84.983,85.046,85.251,85.519,85.716,86.119,86.591,87.188,87.29,87.535,87.901,88.098,88.784,89.638,90.206,90.909,91.413,91.945,92.544,93.258,94.199,95.035,95.347,95.393,95.335,95.267,95.263,95.233,95.174,95.156,95.107,95.212,95.415,95.658,95.896,96.543,97.144,97.242,97.33,97.303,97.202,97.261,98.045,98.446,98.688,98.718,98.812,98.992,99.181,99.529,99.646,99.768,99.834,100,100.39,101.182,101.847,102.756,103.487,103.653,103.623,103.378,103.227,103.176,103.146,103.01,102.618,101.845,100.759,99.582,98.155,95.954,94.159,93.286,92.599,92.216,92.003,91.974,92.053
중구,60.07,60.445,60.613,60.765,60.846,60.877,62.275,62.64,63.024,63.453,63.12,62.936,63.38,63.506,64.275,64.97,65.227,65.958,66.44,66.688,67.072,69.826,71.882,74.318,75.637,76.35,76.643,76.715,77.168,77.24,77.726,77.855,78.355,78.981,78.959,79.229,80.263,80.612,81.128,82.178,83.142,83.637,83.656,83.719,83.719,83.523,82.926,82.314,81.581,82.001,82.444,82.561,82.951,83.328,83.679,83.679,84.549,84.582,84.387,84.387,84.347,84.759,84.931,84.881,84.724,84.597,84.387,84.214,84.142,84.163,84.188,84.347,84.382,84.492,84.606,84.625,84.625,84.625,84.561,84.402,84.382,84.382,84.353,84.214,84.214,84.175,83.497,83.421,82.968,81.558,80.94,80.286,79.228,78.784,78.414,

In [40]:
df_j = df4.loc[df4['건물명'].isin(sett)]
df_j = df_j.dropna(subset = ['층', '면적'])

df_j['법정동코드'] = df_j['법정동코드'].astype(int)
df_j['본번'] = df_j['본번'].astype(int)
df_j['부번'] = df_j['부번'].astype(int)
df_j['면적'] = df_j['면적'].astype(int)
df_j['계약월'] = df_j['계약월'].astype(int)

df_j = df_j.groupby(['자치구명', '법정동명', '본번', '부번', '건물명', '건축년도', '면적', '층', '접수연도', '계약월', '건물용도'])['보증금(만원)'].agg(['mean']).reset_index()
df_j = df_j.rename(columns={'mean': '월별평균 전세가'})
df_j = df_j.loc[~((df_j['접수연도']==2023) & (df_j['계약월']>7))]
df_j['계약월'] = df_j['접수연도'].astype(str) + '-' + df_j['계약월'].astype(str)
df_j['계약월'] = df_j['계약월'].apply(lambda x: tc(x))
df_j.drop(columns=['접수연도'],inplace = True)
df_j['주소'] = df_j['자치구명'] + ' ' + df_j['법정동명'] + ' ' + df_j['본번'].astype(str) + '-' + df_j['부번'].astype(str)
cols_j = df_j.columns.tolist()
cols_j = cols_j[-1:] + cols_j[:-1]
df_j = df_j[cols_j]
df_j.drop(columns=['자치구명', '법정동명', '본번', '부번'],inplace = True)
print(df_j.shape)
display(df_j.head())

(843938, 8)


,주소,건물명,건축년도,면적,층,계약월,건물용도,월별평균 전세가
0,강남구 개포동 12-0,성원대치2단지아파트,1992.0,33,2층 이하,2011-01,아파트,14100.0
1,강남구 개포동 12-0,성원대치2단지아파트,1992.0,33,2층 이하,2011-03,아파트,13000.0
2,강남구 개포동 12-0,성원대치2단지아파트,1992.0,33,2층 이하,2011-04,아파트,12875.0
3,강남구 개포동 12-0,성원대치2단지아파트,1992.0,33,2층 이하,2011-05,아파트,14000.0
4,강남구 개포동 12-0,성원대치2단지아파트,1992.0,33,2층 이하,2011-06,아파트,13700.0


In [41]:
df2 = data_2.copy()
df2.rename(columns={"Unnamed: 1": "자치구명"}, inplace=True)
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
df2.drop([0,1],inplace = True)
df2.drop(['지 역'], axis = 1, inplace = True)
df2 = df2.reset_index(drop=True)
df2 = df2.rename(index = df2['자치구명'])
df2.drop(['자치구명'], axis = 1, inplace = True)
df2.columns = df2.columns.map(tc2)
df2.head()

,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07
종로구,68.513,69.141,70.74,71.407,71.677,71.836,71.987,72.714,74.109,75.204,75.612,75.605,75.605,75.605,75.605,75.796,75.796,75.796,75.84,76.236,76.456,76.554,76.623,76.64,76.709,76.777,77.382,77.822,78.057,78.296,78.731,79.045,79.537,80.065,80.3,80.416,80.919,81.4,81.586,81.711,81.73,81.722,81.718,81.863,82.374,82.618,82.707,82.77,82.913,83.047,83.691,84.221,84.487,84.801,85.095,85.284,85.739,86.185,86.713,86.905,87.02,87.106,87.186,87.318,87.706,87.939,88.414,88.812,89.019,89.44,89.899,90.175,90.336,90.516,90.804,91.115,91.355,91.696,92.166,92.64,92.799,93.012,93.367,93.573,94.095,94.538,95.086,95.24,95.543,96.237,96.561,96.659,97.273,97.383,97.779,97.606,97.443,97.196,96.274,95.882,95.668,95.625,95.545,95.525,95.594,95.731,95.998,96.374,96.813,96.952,97.132,97.222,97.31,97.505,97.766,97.881,98.062,98.219,98.428,99.066,99.505,99.759,99.822,99.883,99.874,100,100.344,101.096,101.77,102.58,103.179,103.379,103.366,103.109,103.002,102.908,102.837,102.704,102.469,101.921,100.899,99.5,97.6,94.593,91.927,89.809,87.845,87.042,86.828,86.909,86.931
중구,64.385,64.881,66.846,69.521,70.042,70.042,71,71.619,72.17,72.44,72.436,72.325,72.325,72.325,71.965,71.965,71.534,70.212,69.962,69.945,69.755,70.019,71.21,71.844,72.07,72.606,73.543,74.475,74.919,75.152,75.551,76.691,77.148,78.308,80.06,80.899,81.546,81.722,82.114,82.185,82.286,82.532,82.85,82.999,83.332,83.947,84.408,84.838,84.98,85.514,86.086,86.591,86.761,87.208,87.454,87.752,88.109,88.71,89.088,89.536,89.619,89.713,89.821,89.935,90.065,90.269,90.528,90.826,90.964,91.181,91.488,91.606,91.703,91.781,91.925,92.127,92.227,92.428,92.804,93.162,93.583,93.878,94.314,94.839,95.016,95.485,95.675,95.614,95.594,95.565,95.667,95.914,96.249,96.778,96.816,96.784,96.595,96.35,96.087,95.984,95.892,95.818,95.688,95.581,95.673,95.832,96.009,96.181,96.483,96.707,96.835,96.848,96.859,96.903,97.113,97.505,97.799,98.085,98.406,99.037,99.459,99.737,99.851,99.897,99.922,100,100.316,100.893,101.623,102.652,103.379,103.541,103.526,103.466,103.31,103.175,103.146,103.083,102.899,102.535,101.555,100.12,97.508,92.519,89.201,87.109,85.555,84.867,84.535,84.398,84.522
용산구,65.28,66.18,66.99,68.634,68.519,68.797,68.973,69.757,71.233,71.457,71.63,71.718,71.718,71.736,71.945,71.967,72.061,72.007,71.995,72.183,72.6,73.099,73.621,73.919,74.286,74.539,75.02,75.973,76.443,76.754,77.091,77.864,78.482,79.49,80.207,80.758,81.286,81.742,82.381,83.007,83.415,83.716,83.949,84.144,84.435,84.648,84.86,85.124,85.476,85.871,86.276,86.841,87.171,87.526,87.932,88.28,88.484,89.031,89.41,89.642,89.841,90.063,90.307,90.492,90.769,90.98,91.305,91.456,91.667,92,92.355,92.576,92.775,92.896,93.099,93.349,93.679,93.8,93.809,93.803,93.821,93.888,93.931,93.909,93.813,93.928,93.962,93.973,93.959,93.988,93.948,94.134,94.255,94.37,94.007,93.611,93.297,92.704,92.435,92

df_m, df1 매매

df_j, df2 전세

In [ ]:
from datetime import datetime, timedelta


# 더미 데이터프레임 생성
data = {
    '주소': [0],
    '건물명': [0],
    '건축년도': [0],
    '면적': [0],
    '층': [0],
    '계약월': [0]
}

In [ ]:
len(list(sett))

In [ ]:
df_dummy = pd.DataFrame(data)

for i in range(len(list(sett))):
    print(i)
    temp = df_m.loc[df_m['건물명'] == list(sett)[i]]
    area = list(set(temp['면적']))
    location = list(set(temp['주소']))
    built_y = list(set(temp['건축년도']))
    floor = list(set(temp['층']))


    for j in area:
        for k in floor:
            for l in location:
                for h in built_y:
                    try:
                        # 시작 날짜와 종료 날짜 설정
                        start_date = datetime.strptime(temp.loc[(temp['면적'] == j) & (temp['층'] == k) & (temp['주소'] == l) & (temp['건축년도'] == h)]['계약월'].min(), '%Y-%m')
                        end_date = datetime.strptime('2023-07', '%Y-%m')

                        # 날짜 범위 생성
                        date_range = pd.date_range(start=start_date, end=end_date + timedelta(days=31), freq='M')  # 월 단위로 생성

                        # 날짜 범위를 컬럼으로 갖는 데이터프레임 생성
                        date_df = pd.DataFrame({'계약월': date_range})
                        date_df['계약월'] = date_df['계약월'].apply(lambda x: str(x)[:7])

                        df_adj = date_df.merge(temp.loc[(temp['면적'] == j) & (temp['층'] == k) & (temp['주소'] == l) & (temp['건축년도'] == h)], how='left', on='계약월')
                        m_data = df1.loc[l.split()[0],list(df_adj['계약월'])]

                        df_adj['건물명'] = list(sett)[i]
                        df_adj['면적'] = j
                        df_adj['층'] = k
                        df_adj['주소'] = l
                        df_adj['건축년도'] = h
                        df_adj['건물용도'] = temp['건물용도'].iloc[0]
                        df_adj['매매지수'] = list(m_data)
                        df_adj['월별평균 매매가'] = df_adj['월별평균 매매가']

                        df_dummy = pd.concat([df_dummy, df_adj])
                    except TypeError:
                        pass

df_m = df_dummy.iloc[1:, :]

df_m.to_csv('매매가평균1.csv', index=False)

In [ ]:
df_dummy = pd.DataFrame(data)

for i in range(len(list(sett))):
    print(i)
    temp = df_j.loc[df_j['건물명'] == list(sett)[i]]
    area = list(set(temp['면적']))
    location = list(set(temp['주소']))
    built_y = list(set(temp['건축년도']))
    floor = list(set(temp['층']))


    for j in area:
        for k in floor:
            for l in location:
                for h in built_y:
                    try:
                        # 시작 날짜와 종료 날짜 설정
                        start_date = datetime.strptime(temp.loc[(temp['면적'] == j) & (temp['층'] == k) & (temp['주소'] == l) & (temp['건축년도'] == h)]['계약월'].min(), '%Y-%m')
                        end_date = datetime.strptime('2023-07', '%Y-%m')

                        # 날짜 범위 생성
                        date_range = pd.date_range(start=start_date, end=end_date + timedelta(days=31), freq='M')  # 월 단위로 생성

                        # 날짜 범위를 컬럼으로 갖는 데이터프레임 생성
                        date_df = pd.DataFrame({'계약월': date_range})
                        date_df['계약월'] = date_df['계약월'].apply(lambda x: str(x)[:7])

                        df_adj = date_df.merge(temp.loc[(temp['면적'] == j) & (temp['층'] == k) & (temp['주소'] == l) & (temp['건축년도'] == h)], how='left', on='계약월')
                        m_data = df2.loc[l.split()[0],list(df_adj['계약월'])]

                        df_adj['건물명'] = list(sett)[i]
                        df_adj['면적'] = j
                        df_adj['층'] = k
                        df_adj['주소'] = l
                        df_adj['건축년도'] = h
                        df_adj['건물용도'] = temp['건물용도'].iloc[0]
                        df_adj['전세지수'] = list(m_data)
                        df_adj['월별평균 전세가'] = df_adj['월별평균 전세가']

                        df_dummy = pd.concat([df_dummy, df_adj])
                    except TypeError:
                        pass

df_j = df_dummy.iloc[1:, :]

df_j.to_csv('전세가평균1.csv', index=False)

In [ ]:
data_mm = pd.read_csv('매매가평균1.csv')
data_jj = pd.read_csv('전세가평균1.csv')

In [ ]:
display(data_mm[data_mm['월별평균 매매가'].isin([0])])
display(data_jj[data_jj['월별평균 전세가'].isin([0])])

In [ ]:
df_mm = data_mm.copy()
df_mm['월별평균 매매가'] = df_mm['월별평균 매매가'] / df_mm['매매지수']
df_mm = df_mm.fillna(method="ffill")
df_mm['월별평균 매매가'] = df_mm['월별평균 매매가'] * df_mm['매매지수']
df_mm

In [ ]:
df_jj = data_jj.copy()
df_jj['월별평균 전세가'] = df_jj['월별평균 전세가'] / df_jj['전세지수']
df_jj = df_jj.fillna(method="ffill")
df_jj['월별평균 전세가'] = df_jj['월별평균 전세가'] * df_jj['전세지수']
df_jj

In [ ]:
df_mm.to_csv('매매가평균2.csv', index=False)

In [ ]:
df_jj.to_csv('전세가평균2.csv', index=False)

In [20]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime as dt

data_mm = pd.read_csv('매매가평균1.csv')

In [22]:
df_mm = data_mm.copy()
df_mm = df_mm.drop(columns=['매매지수'])

df_mm['건물'] = df_mm['주소'] + ' ' + df_mm['건물용도']
df_mm['건물'] = df_mm['건물'] + ' ' + df_mm['건물명']
df_mm['건물'] = df_mm['건물'] + '(' + df_mm['건축년도'].astype(str) + ')'
df_mm['건물'] = df_mm['건물'] + ' ' + df_mm['면적'].astype(str) + '㎡'
df_mm['건물'] = df_mm['건물'] + ' ' + df_mm['층']
df_mm = df_mm.drop(columns=['주소', '건물명', '건축년도', '면적', '층', '건물용도'])
cols_df_mm = df_mm.columns.tolist()
cols_df_mm = cols_df_mm[-1:] + cols_df_mm[:-1]
df_mm = df_mm[cols_df_mm]
df_mm = df_mm.groupby(['건물', '계약월']).mean().reset_index()
df_mm = df_mm.sort_values(by=['건물', '계약월'], ascending=True).reset_index(drop=True)
df_mm

,건물,계약월,월별평균 매매가
0,강남구 개포동 1164-12 아파트 새롬(1164-12)(2000) 72㎡ 2층 초과,2017-09,64500.0
1,강남구 개포동 1164-12 아파트 새롬(1164-12)(2000) 72㎡ 2층 초과,2017-10,NaN
2,강남구 개포동 1164-12 아파트 새롬(1164-12)(2000) 72㎡ 2층 초과,2017-11,NaN
3,강남구 개포동 1164-12 아파트 새롬(1164-12)(2000) 72㎡ 2층 초과,2017-12,NaN
4,강남구 개포동 1164-12 아파트 새롬(1164-12)(2000) 72㎡ 2층 초과,2018-01,NaN
...,...,...,...
5541916,중랑구 중화동 454-0 아파트 동양엔파트2차(2013) 96㎡ 2층 초과,2023-03,NaN
5541917,중랑구 중화동 454-0 아파트 동양엔파트2차(2013) 96㎡ 2층 초과,2023-04,NaN
5541918,중랑구 중화동 454-0 아파트 동양엔파트2차(2013) 96㎡ 2층 초과,2023-05,NaN
5541919,중랑구 중화동 454-0 아파트 동양엔파트2차(2013) 96㎡ 2층 초과,2023-06,NaN


In [23]:
start_date = dt.strptime(df_mm['계약월'].min(), '%Y-%m')
end_date = dt.strptime('2010-12', '%Y-%m')
print(start_date, end_date)
df_mm[df_mm['계약월']<'2011-01-01']['계약월'].unique()

2011-01-01 00:00:00 2010-12-01 00:00:00


array(['2011-01'], dtype=object)

In [9]:
ts1_mm = sorted(df_mm['계약월'].unique().tolist())

df_ts_mm = pd.DataFrame(columns = ts1_mm)

i = 0
for x in df_mm['건물'].unique():
    i += 1
    print(i,x)
    df_dummy = df_mm[df_mm['건물']==x]
    df_dummy_mm = pd.DataFrame(data = np.array([df_dummy['월별평균 매매가']]), columns = df_dummy['계약월'].values, index = [x])
    df_ts_mm = pd.concat([df_ts_mm, df_dummy_mm])

1 강남구 개포동 1164-12 아파트 새롬(1164-12)(2000) 72㎡ 2층 초과
2 강남구 개포동 1164-12 아파트 새롬(1164-12)(2000) 74㎡ 2층 이하
3 강남구 개포동 1164-12 아파트 새롬(1164-12)(2000) 74㎡ 2층 초과
4 강남구 개포동 1164-13 아파트 새롬(1164-13)(2000) 60㎡ 2층 이하
5 강남구 개포동 1164-13 아파트 새롬(1164-13)(2000) 60㎡ 2층 초과
6 강남구 개포동 1164-14 아파트 새롬(1164-14)(2000) 40㎡ 2층 이하
7 강남구 개포동 1164-14 아파트 새롬(1164-14)(2000) 40㎡ 2층 초과
8 강남구 개포동 1164-14 아파트 새롬(1164-14)(2000) 58㎡ 2층 이하
9 강남구 개포동 1164-14 아파트 새롬(1164-14)(2000) 58㎡ 2층 초과
10 강남구 개포동 1164-20 아파트 한림(2002) 73㎡ 2층 이하
11 강남구 개포동 1164-20 아파트 한림(2002) 73㎡ 2층 초과
12 강남구 개포동 1164-25 아파트 리더스(2002) 84㎡ 2층 초과
13 강남구 개포동 1164-28 아파트 포이벨리(2001) 85㎡ 2층 이하
14 강남구 개포동 1164-28 아파트 포이벨리(2001) 85㎡ 2층 초과
15 강남구 개포동 1164-30 아파트 뉴현대파크빌(2001) 69㎡ 2층 초과
16 강남구 개포동 1164-30 아파트 뉴현대파크빌(2001) 70㎡ 2층 초과
17 강남구 개포동 1164-30 아파트 뉴현대파크빌(2001) 74㎡ 2층 이하
18 강남구 개포동 1164-30 아파트 뉴현대파크빌(2001) 74㎡ 2층 초과
19 강남구 개포동 1164-30 아파트 뉴현대파크빌(2001) 79㎡ 2층 초과
20 강남구 개포동 1164-30 아파트 뉴현대파크빌(2001) 83㎡ 2층 이하
21 강남구 개포동 1164-7 아파트 아람손프라자/비바빌(2001) 60㎡ 2층 초과
22 강남구 개포동

KeyboardInterrupt: 

In [ ]:
df_ts_mm.to_csv('매매가시계열.csv')

In [ ]:
df_mm = df_mm.drop(columns=['매매지수'])
display(df_mm.head())
df_jj = df_jj.drop(columns=['전세지수'])
df_jj['건축년도'] = df_jj['건축년도'].astype(int)
display(df_jj.head())

In [ ]:
df = pd.merge(df_mm, df_jj, how="inner", on=['주소', '건물명', '건축년도', '면적', '층', '계약월', '건물용도'])

df['건물'] = df['주소'] + ' ' + df['건물용도']
df['건물'] = df['건물'] + ' ' + df['건물명']
df['건물'] = df['건물'] + '(' + df['건축년도'].astype(str) + ')'
df['건물'] = df['건물'] + ' ' + df['면적'].astype(str) + '㎡'
df['건물'] = df['건물'] + ' ' + df['층']
df = df.drop(columns=['주소', '건물명', '건축년도', '면적', '층', '건물용도'])

cols_df = df.columns.tolist()
cols_df = cols_df[-1:] + cols_df[:-1]
df = df[cols_df]
df = df.groupby(['건물', '계약월']).mean().reset_index()
df = df.sort_values(by=['건물', '계약월'], ascending=True).reset_index()
df

In [ ]:
ts1 = sorted(df['계약월'].unique().tolist())

df_ts = pd.DataFrame(columns = ts1)

df_ts

In [ ]:
df_ts_m = df_ts.copy()
df_ts_j = df_ts.copy()
i = 0
for x in df['건물'].unique():
    i += 1
    print(i,x)
    df_dummy = df[df['건물']==x]
    df_dummy_m = pd.DataFrame(data = np.array([df_dummy['월별평균 매매가']]), columns = df_dummy['계약월'].values, index = [x])
    df_dummy_j = pd.DataFrame(data = np.array([df_dummy['월별평균 전세가']]), columns = df_dummy['계약월'].values, index = [x])
    df_ts_m = pd.concat([df_ts_m, df_dummy_m])
    df_ts_j = pd.concat([df_ts_j, df_dummy_j])

In [ ]:
df_ts_m.to_csv('매매가시계열.csv')

In [ ]:
df_ts_j.to_csv('전세가시계열.csv')

In [ ]:
df_ts_m

In [ ]:
df_ts_j

In [10]:
data_mm = pd.read_csv('매매가평균2.csv')
data_jj = pd.read_csv('전세가평균2.csv')

In [11]:
data_mm

,주소,건물명,건축년도,면적,층,계약월,건물용도,월별평균 매매가,매매지수
0,구로구 가리봉동 137-7,미래하이츠,2003,97,2층 초과,2016-02,아파트,32300.000000,80.879
1,구로구 가리봉동 137-7,미래하이츠,2003,97,2층 초과,2016-03,아파트,32272.044659,80.809
2,구로구 가리봉동 137-7,미래하이츠,2003,97,2층 초과,2016-04,아파트,32307.987240,80.899
3,구로구 가리봉동 137-7,미래하이츠,2003,97,2층 초과,2016-05,아파트,32364.297284,81.040
4,구로구 가리봉동 137-7,미래하이츠,2003,97,2층 초과,2016-06,아파트,32461.342252,81.283
...,...,...,...,...,...,...,...,...,...
5542048,종로구 명륜3가 1-102,(1-102),1997,70,2층 이하,2023-03,아파트,20770.931715,92.599
5542049,종로구 명륜3가 1-102,(1-102),1997,70,2층 이하,2023-04,아파트,20685.020778,92.216
5542050,종로구 명륜3가 1-102,(1-102),1997,70,2층 이하,2023-05,아파트,20637.242633,92.003
5542051,종로구 명륜3가 1-102,(1-102),1997,70,2층 이하,2023-06,아파트,20630.737628,91.974


In [12]:
data_jj

,주소,건물명,건축년도,면적,층,계약월,건물용도,월별평균 전세가,전세지수
0,구로구 가리봉동 137-7,미래하이츠,2003.0,84,2층 이하,2016-06,아파트,26000.000000,88.176
1,구로구 가리봉동 137-7,미래하이츠,2003.0,84,2층 이하,2016-07,아파트,26143.599165,88.663
2,구로구 가리봉동 137-7,미래하이츠,2003.0,84,2층 이하,2016-08,아파트,26250.929958,89.027
3,구로구 가리봉동 137-7,미래하이츠,2003.0,84,2층 이하,2016-09,아파트,26326.710216,89.284
4,구로구 가리봉동 137-7,미래하이츠,2003.0,84,2층 이하,2016-10,아파트,26436.105063,89.655
...,...,...,...,...,...,...,...,...,...
5196693,종로구 명륜3가 1-102,(1-102),1997.0,70,2층 이하,2023-03,아파트,16309.204555,87.845
5196694,종로구 명륜3가 1-102,(1-102),1997.0,70,2층 이하,2023-04,아파트,16160.120472,87.042
5196695,종로구 명륜3가 1-102,(1-102),1997.0,70,2층 이하,2023-05,아파트,16120.389471,86.828
5196696,종로구 명륜3가 1-102,(1-102),1997.0,70,2층 이하,2023-06,아파트,16135.427841,86.909


print('-'*130)
print('-'*130)
print('-'*130)
print('-'*130)

df = pd.merge(df3[['접수연도', '자치구코드', '자치구명', '법정동코드', '법정동명', '지번구분', '지번구분명', '본번', '부번','층','면적','계약월','평균물건금액(만원)']], df4, how='inner', on=['접수연도', '자치구코드', '자치구명', '법정동코드', '법정동명', '지번구분', '지번구분명', '본번', '부번',
        '층','면적','계약월'])

df = df[~df['평균물건금액(만원)'].isnull()].reset_index(drop=True)

df = df.sort_values(by=['접수연도'],ascending = True)

df = df.sort_values(by=['자치구명', '법정동명', '층', '건물명', '면적'])

df.drop(columns=['자치구명', '법정동명', '지번구분명', '전월세 구분', '계약기간', '신규갱신여부', '종전 보증금', '계약일', '종전 임대료', '계약개월', '계약갱신권사용여부', '건물명'],inplace = True)

df.fillna(2004,inplace = True)

df['법정동코드'] = (df['자치구코드'].astype(str) + df['법정동코드'].astype(str)).astype(int)

df['지번구분'] = df['지번구분'].astype(int)

df['본번'] = df['본번'].astype(int)

df['부번'] = df['부번'].astype(int)

df['층'] = df['층'].astype(int)

df['면적'] = df['면적'].astype(int)

df['건축년도'] = df['건축년도'].astype(int)

df['전세가율'] = df['보증금(만원)'] / df['평균물건금액(만원)'] * 100

df = df.drop_duplicates().reset_index(drop=True)

df['깡통전세'] = df['전세가율']>75

df.drop(columns=['전세가율'],inplace = True)

df['깡통전세'] = df['깡통전세'].astype(int)

df

from sklearn.model_selection import train_test_split


y = df['깡통전세']
X = df.drop('깡통전세', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

print("X_train shape: ", X_train.shape)
print("X_val shape: ", X_val.shape)
print("X_test shape: ", X_test.shape)

from sklearn.metrics import accuracy_score

base_major = y_train.mode()[0]

y_pred_base = [base_major] * len(y_train)

print("training accuracy: ", accuracy_score(y_train, y_pred_base).round(2))

y_pred_base_val = [base_major] * len(y_val)

print("validation accuracy: ", accuracy_score(y_val, y_pred_base_val).round(2))

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

numeric_feats = X_train.dtypes[X_train.dtypes != "object"].index

scaler = MinMaxScaler()
X_train[numeric_feats] = scaler.fit_transform(X_train[numeric_feats])
X_val[numeric_feats] = scaler.transform(X_val[numeric_feats])
X_test[numeric_feats] = scaler.transform(X_test[numeric_feats])

ohe = OneHotEncoder(handle_unknown='ignore')

X_train_ohe = ohe.fit_transform(X_train)
X_val_ohe = ohe.transform(X_val)
X_test_ohe = ohe.transform(X_test)

from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression, LogisticRegression

logistic = LogisticRegression(class_weight='balanced')
logistic.fit(X_train_ohe, y_train)
y_val_pred = logistic.predict(X_val_ohe)
print('logistic train accuracy', logistic.score(X_train_ohe, y_train).round(2))
print("logistic validation accuracy: ", accuracy_score(y_val, y_val_pred).round(2))

from sklearn.metrics import classification_report, recall_score, precision_score, f1_score

print(classification_report(y_val, y_val_pred))

accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print(f'accuracy of logistic regression is {accuracy.round(2)}')
print(f'precision of logistic regression is {precision.round(2)}')
print(f'recall of logistic regression is {recall.round(2)}')
print(f'f1 of logistic regression is {f1.round(2)}')

y_test_pred = logistic.predict(X_test_ohe)

print(classification_report(y_test, y_test_pred))

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print(f'accuracy of logistic regression is {accuracy.round(2)}')
print(f'precision of logistic regression is {precision.round(2)}')
print(f'recall of logistic regression is {recall.round(2)}')
print(f'f1 of logistic regression is {f1.round(2)}')